In [1]:
from pysheds.grid import Grid
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
from PIL import Image
import os
import rasterio
from rasterio.windows import Window
import time
from skimage.io import imread
from PIL import Image

In [2]:
def sketch_rivers(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    depressions = grid.detect_depressions('dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    flats = grid.detect_flats('flooded_dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    # Compute flow direction based on corrected DEM
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Compute flow accumulation based on computed flow direction
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    
    downsampled_rivers = np.log(grid.view('acc') + 1)
    upsampled_rivers = cv2.pyrUp(downsampled_rivers, dstsize=(512, 512))
    upsampled_rivers = (upsampled_rivers - np.amin(upsampled_rivers)) / (np.amax(upsampled_rivers) - np.amin(upsampled_rivers))
    upsampled_rivers = np.array(upsampled_rivers * 255, dtype=np.uint8)
    _, thresholded_river = cv2.threshold(upsampled_rivers, 127, 255, cv2.THRESH_BINARY)
    thresholded_river[thresholded_river == 255] = 1
    skeletonized_rivers = skeletonize(thresholded_river)
    rivers_sketch = Image.fromarray(skeletonized_rivers)
    rgb_rivers_sketch = rivers_sketch.convert('RGB')

    upsampled_depressions = cv2.pyrUp(np.array(depressions, dtype=np.uint8), dstsize=(512, 512))
    upsampled_depressions = (upsampled_depressions - np.amin(upsampled_depressions)) / (np.amax(upsampled_depressions) - np.amin(upsampled_depressions))
    upsampled_depressions = np.array(upsampled_depressions * 255, dtype=np.uint8)
    _, thresholded_depressions = cv2.threshold(upsampled_depressions, 127, 255, cv2.THRESH_BINARY)
    basins_sketch = Image.fromarray(thresholded_depressions)
    rgb_basins_sketch = basins_sketch.convert('RGB')
    
    return rgb_rivers_sketch, rgb_basins_sketch

In [3]:
def sketch_ridges(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.dem = grid.dem.max() - grid.dem
    peaks = grid.detect_depressions('dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    flats = grid.detect_flats('flooded_dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    # Compute flow direction based on corrected DEM
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    # Compute flow accumulation based on computed flow direction
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    downsampled_ridges = np.log(grid.view('acc') + 1)
    
    upsampled_ridges = cv2.pyrUp(downsampled_ridges, dstsize=(512, 512))
    upsampled_ridges = (upsampled_ridges - np.amin(upsampled_ridges)) / (np.amax(upsampled_ridges) - np.amin(upsampled_ridges))
    upsampled_ridges = np.array(upsampled_ridges * 255, dtype=np.uint8)
    _, thresholded_ridges = cv2.threshold(upsampled_ridges, 150, 255, cv2.THRESH_BINARY)
    thresholded_ridges[thresholded_ridges == 255] = 1
    skeletonized_ridges = skeletonize(thresholded_ridges)
    ridges_sketch = Image.fromarray(skeletonized_ridges)
    rgb_ridges_sketch = ridges_sketch.convert('RGB')
    
    upsampled_peaks = cv2.pyrUp(np.array(peaks, dtype=np.uint8), dstsize=(512, 512))
    upsampled_peaks = (upsampled_peaks - np.amin(upsampled_peaks)) / (np.amax(upsampled_peaks) - np.amin(upsampled_peaks))
    upsampled_peaks = np.array(upsampled_peaks * 255, dtype=np.uint8)
    _, thresholded_peaks = cv2.threshold(upsampled_peaks, 127, 255, cv2.THRESH_BINARY)
    peaks_sketch = Image.fromarray(thresholded_peaks)
    rgb_peaks_sketch = peaks_sketch.convert('RGB')
    
    return rgb_ridges_sketch, rgb_peaks_sketch

In [4]:
def prepare_heightmap_sketch(file_path):
    rivers, basins = sketch_rivers(file_path)
    ridges, peaks = sketch_ridges(file_path)

    dim = (512, 512)

    sketch = np.zeros([512,512,3],dtype=np.uint8)
    sketch.fill(255)

    pixels_rivers = rivers.load() # create the pixel map
    
    for i in range(rivers.size[0]): # for every pixel:
        for j in range(rivers.size[1]):
            if pixels_rivers[i,j] == (255, 255, 255):
                sketch[j,i] = (0, 0, 255)

            
    pixels_basins = basins.load() # create the pixel map
            
    for i in range(basins.size[0]): # for every pixel:
        for j in range(basins.size[1]):
            if pixels_basins[i,j] == (255, 255, 255):
                sketch[j,i] = (0, 255, 0)    
            
            
    pixels_ridges = ridges.load() # create the pixel map
            
    for i in range(ridges.size[0]): # for every pixel:
        for j in range(ridges.size[1]):
            if pixels_ridges[i,j] == (255, 255, 255):
                sketch[j,i] = (255, 0, 0) 
            
            
    pixels_peaks = peaks.load() # create the pixel map

    for i in range(peaks.size[0]): # for every pixel:
        for j in range(peaks.size[1]):
            if pixels_peaks[i,j] == (255, 255, 255):
                sketch[j,i] = (0, 0, 0)   
            
    sketch_image = Image.fromarray(sketch)
    
    return sketch_image

In [5]:
# GeoTIFF from NASA src path

topoPathA1 = 'data/topography_10800x10800_A1.tif'
topoPathA2 = 'data/topography_10800x10800_A2.tif'
topoPathB1 = 'data/topography_10800x10800_B1.tif'
topoPathB2 = 'data/topography_10800x10800_B2.tif'
topoPathC1 = 'data/topography_10800x10800_C1.tif'
topoPathC2 = 'data/topography_10800x10800_C2.tif'
topoPathD1 = 'data/topography_10800x10800_D1.tif'
topoPathD2 = 'data/topography_10800x10800_D2.tif'

topoArrayPath = []
topoArrayPath.append(topoPathA1)
topoArrayPath.append(topoPathA2)
topoArrayPath.append(topoPathB1)
topoArrayPath.append(topoPathB2)
topoArrayPath.append(topoPathC1)
topoArrayPath.append(topoPathC2)
topoArrayPath.append(topoPathD1)
topoArrayPath.append(topoPathD2)

# Size of tiles
xSize, ySize = 256, 256

In [6]:
# returns True if we should keep heightmap tilek, otherwise False to discard it
# counts how much black in heightmap (to discard only water areas)
def comparator(heightmap_chunk):
    black_ratio = ((heightmap_chunk==0).sum()*1.0)/float(np.prod(heightmap_chunk.shape))
    if black_ratio > 0.9:
        return False
    return True

In [8]:
src_list, tar_list = list(), list()
savePathName = 'topography_data.tif'
dim = (512, 512)
iterator = 0
# Cut geotiff images and save to folder, remove images where 90% area is water

for srcTifPath in topoArrayPath:
    t0 = time.time()
    iterator = 0
    with rasterio.open(srcTifPath) as src:  
        for y in range(0, src.width, 5000):
            for x in range(0, src.height, 5000):
                # Create a Window and calculate the transform from the source dataset    
                window = Window(x, y, xSize, ySize)
                transform = src.window_transform(window)

                # Create a new cropped raster to write to
                profile = src.profile
                profile.update({
                    'height': xSize,
                    'width': ySize,
                    'transform': transform})
                        
                with rasterio.open(savePathName, 'w', **profile) as dst:
                    # Read the data from the window and write it to the output raster
                    dst.write(src.read(window=window))
                    
                texture = imread(savePathName)
            
                if comparator(texture):
                    iterator += 1
                    sketch = prepare_heightmap_sketch(savePathName)
                    heightmap = cv2.imread(savePathName)
        
                    sketch_array = np.array(sketch)
                    heightmap_array = np.array(Image.fromarray((cv2.resize(np.array(heightmap), dim, interpolation = cv2.INTER_AREA)), 'RGB'))
        
                    src_list.append(sketch_array)
                    tar_list.append(heightmap_array)
                
                os.remove(savePathName)

    print(str(time.time()-t0) + 's for ' + srcTifPath + '. Saved ' + str(iterator) + ' images')
    
np.savez_compressed('network_data.npz', np.asarray(src_list), np.asarray(tar_list))

3.5249640941619873s for data/topography_10800x10800_A1.tif. Saved 1 images
9.276004076004028s for data/topography_10800x10800_A2.tif. Saved 3 images
6.176031827926636s for data/topography_10800x10800_B1.tif. Saved 2 images
12.397995710372925s for data/topography_10800x10800_B2.tif. Saved 4 images
15.440000772476196s for data/topography_10800x10800_C1.tif. Saved 5 images
9.726073741912842s for data/topography_10800x10800_C2.tif. Saved 3 images
3.5570592880249023s for data/topography_10800x10800_D1.tif. Saved 2 images
16.29679250717163s for data/topography_10800x10800_D2.tif. Saved 5 images
